Aim: pull unWISE PSFs using the PSFs associated with COADD tiles.

* Match centers of galaxies with nearest coadd tile
* Create a catalog column which includes the coadd IDs
* Run unwise_psf.py to extract correct PSF with the coadd ID
* Save PSF image for each galaxy

In [81]:
#import libraries
from astropy.table import Table, vstack, unique
from astropy.io import fits
import numpy as np
import os

homedir=os.getenv("HOME")

### Part One: Matching Galaxies to Coadd Tiles

In [11]:
#read in tiles.fits, which lists tile coadd_ids and the corresponding (RA, DEC) of the tiles' centers
tilefile=Table.read(homedir+'/github/wisesize/unwise_PSFs/tiles.fits')

In [12]:
#let's briefly look at the format of the table...
tilefile

coadd_id,ra,dec,row
bytes8,float64,float64,int64
0000m016,0.0,-1.514444,0
0000m031,0.0,-3.028889,1
0000m046,0.0,-4.543333,2
0000m061,0.0,-6.057778,599
0000m076,0.0,-7.572222,600
0000m091,0.0,-9.086667,601
0000m107,0.0,-10.601111,602
0000m122,0.0,-12.115556,603
0000m137,0.0,-13.63,604


In [68]:
#yay. now, load the galaxy catalog of choice. for my purposes, I will use my (preliminary) WISESize table
wisesize = Table.read(homedir+'/Desktop/wisesize/wisesize_v1.fits')

#isolate RA and DEC columns
wise_RA = wisesize['RA']
wise_DEC = wisesize['DEC']

tile_RA = tilefile['ra']
tile_DEC = tilefile['dec']

In [75]:
#create empty index array
idx_arr=np.zeros(len(RA),dtype=int)

#for every catalog RA and DEC, find tilefile index where the central RA&DEC most closely matches to catalog RA&DEC
for n in range(len(wise_RA)):
    idx = (np.abs(tile_RA - wise_RA[n]) + np.abs(tile_RA - wise_DEC[n])).argmin()
    idx_arr[n]=idx

In [78]:
#set new column (coadd_id) which contains the appropriate row-matched tile name for each galaxy
wisesize['coadd_id'] = tilefile[idx_arr]['coadd_id']

### Part Two: Pulling PSF Images Based on Coadd IDs

In [104]:
#we now have a table of sample galaxies and their corresponding Coadd IDs. What about the unWISE PSFs?
#firstly...we need to pull a github repository
        #https://github.com/legacysurvey/unwise_psf.git

#unwise_psf.py has some wonky dependencies, namely astrometry.net
#we need to git clone that repository as well
#https://github.com/dstndstn/astrometry.net.git
#be sure that your python library contains the astrometry.net dependencies as explained in the README.md file

#one workaround to ensure you can access what you need (without having to BUILD astrometry.net) 
#begins with a simple pip install
#pip install astrometry

#cd to where you git cloned astrometry.net, and locate the util directory
#move (or cp) the util directory from astrometry.net repository to astrometry repository (the path to which
#you can find by typing the following into a terminal window:
#python -c "import astrometry; print(astrometry.__file__)"

#NOTE: if you would rather build astrometry.net but have trouble importing the libraries when
#running python, try typing in a terminal window
#export PYTHONPATH=${PYTHONPATH}:$CONDA_PREFIX/lib/python
#because the astrometry.net code by default installs the code in a place where python won't find it.

In [105]:
#move to location where you git cloned the repository
os.chdir(homedir+'/github/unwise_psf/py')
import unwise_psf.unwise_psf as unwise_psf

In [106]:
#examples:
#psf_w1 = unwise_psf.get_unwise_psf(1,'0000m016')
#psf_w2 = unwise_psf.get_unwise_psf(2, '0000m016')
#psf_w3 = unwise_psf.get_unwise_psf(3, '0000m016')
#psf_w4 = unwise_psf.get_unwise_psf(4, '0000m016')

#the output will be a 2D data array of the relevant PSF, which you can then write to a filename
#according to a naming convention of your choice.

In [111]:
#for my purposes...
#on draco VMS, data is mounted at
#/mnt/astrophysics/lsdr9-wisesize/dr9-_____/native/
#where _____ = north or south
#each directory in native is labeled as a 3-digit RA integer (e.g., 090, 100, 283). 

#each filename follows this type of scheme:
#SGA2025_J090.6918+60.5232.fits

#[090.6918] is the RA rounded to 4 sigfigs, [+60.5232] is the DEC rounded to 4 sigfigs

#I could probably take each filename, append '-PSF-W3.fits', and proceed from there.
#e.g., SGA2025_J090.6918+60.5232-PSF-W3.fits

In [156]:
for n in range(1,5):
    print(n)

1
2
3
4
